# Arithmetic operations

In [1]:
import torch
import math
import numpy as np

In [2]:
x = torch.ones(2, 3)

### matrix product

In [3]:
y1 = x @ x.T
y2 = x.matmul(x.T)

y3 = torch.zeros((2, 2))
torch.matmul(x, x.T, out=y3)
print(y1,'\n', y2,'\n', y3)

tensor([[3., 3.],
        [3., 3.]]) 
 tensor([[3., 3.],
        [3., 3.]]) 
 tensor([[3., 3.],
        [3., 3.]])


### element-wise product

In [4]:
z1 = x * x
z2 = x.mul(x)

z3 = torch.rand_like(x)
torch.mul(x, x, out=z3)
print(z1,'\n', z2,'\n', z3)

tensor([[1., 1., 1.],
        [1., 1., 1.]]) 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 
 tensor([[1., 1., 1.],
        [1., 1., 1.]])


### 从单元素tensor中提取元素的value

In [5]:
y = torch.sum(x) # or y = x.sum()
value_y = y.item()
print(y, type(y), value_y, type(value_y))

tensor(6.) <class 'torch.Tensor'> 6.0 <class 'float'>


### broadcast
规则：按照**last to first**的顺序比较每个dim的值，
所有dim值相同的话，直接运算；\
不相同的dim上，其中一个operand的dim值是0；\
或者，dimension在其中一个tensor上不存在。

In [6]:
a =     torch.ones(2, 3, 2)
b = a * torch.rand(   3, 2) # 3rd & 2nd dims identical to a, dim 1 absent
print(b)

c = a * torch.rand(   3, 1) # 3rd dim = 1, 2nd dim identical to a
print(c)

d = a * torch.rand(   1, 2) # 3rd dim identical to a, 2nd dim = 1
print(d)

tensor([[[0.7664, 0.9773],
         [0.8280, 0.1166],
         [0.1588, 0.2164]],

        [[0.7664, 0.9773],
         [0.8280, 0.1166],
         [0.1588, 0.2164]]])
tensor([[[0.5639, 0.5639],
         [0.6936, 0.6936],
         [0.9699, 0.9699]],

        [[0.5639, 0.5639],
         [0.6936, 0.6936],
         [0.9699, 0.9699]]])
tensor([[[0.2750, 0.1434],
         [0.2750, 0.1434],
         [0.2750, 0.1434]],

        [[0.2750, 0.1434],
         [0.2750, 0.1434],
         [0.2750, 0.1434]]])


In [7]:
# 典型错误
a =     torch.ones(4, 3, 2)
# b = a * torch.rand(4, 3)    # dimensions must match last-to-first
# c = a * torch.rand(   2, 3) # both 3rd & 2nd dims different
# d = a * torch.rand((0, ))   # can't broadcast with an empty tensor

### in-place ops
大多数binary operations都会返回新建的tensor，但很多数学运算也支持in-place operation：
1. 用下划线标记，或者用函数提供的out参数
2. 虽然in-place操作可以节省存储空间，但求梯度的时候不要用，会丢失weights history

In [8]:
x = torch.rand((2, 3))
print('原x:\n', x)
x.add_(1)
print('in-place add:\n', x)
x.t_()
print('in-place transpose:\n', x)

原x:
 tensor([[0.7349, 0.1914, 0.1015],
        [0.8505, 0.9423, 0.9360]])
in-place add:
 tensor([[1.7349, 1.1914, 1.1015],
        [1.8505, 1.9423, 1.9360]])
in-place transpose:
 tensor([[1.7349, 1.8505],
        [1.1914, 1.9423],
        [1.1015, 1.9360]])


In [12]:
a = torch.tensor([0, math.pi / 4, math.pi / 2, 3 * math.pi / 4])
print('a:\n', a)
print("output:\n", torch.sin(a)) 
print("a没变：\n", a) 

b = torch.tensor([0, math.pi / 4, math.pi / 2, 3 * math.pi / 4])
print('b:\n', b)
print("output:\n", torch.sin_(b))
print("b改变：\n", b) 

a:
 tensor([0.0000, 0.7854, 1.5708, 2.3562])
output:
 tensor([0.0000, 0.7071, 1.0000, 0.7071])
a没变：
 tensor([0.0000, 0.7854, 1.5708, 2.3562])
b:
 tensor([0.0000, 0.7854, 1.5708, 2.3562])
output:
 tensor([0.0000, 0.7071, 1.0000, 0.7071])
b改变：
 tensor([0.0000, 0.7071, 1.0000, 0.7071])


In [13]:
a = torch.rand(2, 2)
b = torch.rand(2, 2)
c = torch.zeros(2, 2)
old_id = id(c)

print(c)
d = torch.matmul(a, b, out=c)
print(c)                # contents of c have changed

assert c is d           # test c & d are same object, not just containing equal values
assert id(c) == old_id  # make sure that our new c is the same object as the old one

torch.rand(2, 2, out=c) # works for creation too!
print(c)                # c has changed again
assert id(c) == old_id  # still the same object!

tensor([[0., 0.],
        [0., 0.]])
tensor([[0.4293, 0.2532],
        [0.8134, 0.5592]])
tensor([[0.9239, 0.9773],
        [0.4088, 0.9971]])


### copy tensor
1. assignment只是给等号右边对象新建了一个label，并没有复制新的对象
2. 如果要复制，可以用tensor.clone()
3. 如果被复制的tensor enabled autograd，那么clone得到的tensor也会enable。如果不希望clone得到的tensor复制原tensor的autograd设置，可以用tensor.detach()

In [19]:
a = torch.ones(2, 2)
b = a.clone()

assert b is not a      # 在memory中是不同的object
# assert b == a          # 不能这样assert，因为b==a输出的是boolean矩阵，assert只处理单个boolean值
print(torch.eq(a, b))  # ...but still with the same contents!

a[0][1] = 561          # a changes...
print(b)               # ...but b is still all ones

tensor([[True, True],
        [True, True]])
tensor([[1., 1.],
        [1., 1.]])


In [20]:
a = torch.rand(2, 2, requires_grad=True) # turn on autograd
print(a)

b = a.clone()
print(b)

c = a.detach().clone()  # 这里在不改变a的设置的条件下，关闭c的autograd
print(c)                # c的autograd关闭
print(a)                # a的autograd还是开启状态

tensor([[0.6986, 0.0886],
        [0.3698, 0.3061]], requires_grad=True)
tensor([[0.6986, 0.0886],
        [0.3698, 0.3061]], grad_fn=<CloneBackward0>)
tensor([[0.6986, 0.0886],
        [0.3698, 0.3061]])
tensor([[0.6986, 0.0886],
        [0.3698, 0.3061]], requires_grad=True)
